#Flatten the json

In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [0]:
%fs ls "/FileStore/tables/"

path,name,size,modificationTime
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121,1716379200000
dbfs:/FileStore/tables/Bucketing_by_id/,Bucketing_by_id/,0,0
dbfs:/FileStore/tables/Corruptedjson.json,Corruptedjson.json,222,1716465677000
dbfs:/FileStore/tables/File.csv,File.csv,446,1716524311000
dbfs:/FileStore/tables/employee.csv,employee.csv,236,1716443570000
dbfs:/FileStore/tables/file5.json,file5.json,669503,1716466304000
dbfs:/FileStore/tables/linedelimited.json,linedelimited.json,219,1716465123000
dbfs:/FileStore/tables/multilinejson.json,multilinejson.json,310,1716465123000
dbfs:/FileStore/tables/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz.parquet,part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz.parquet,3921,1716469909000
dbfs:/FileStore/tables/partition_by_address_gender/,partition_by_address_gender/,0,0


In [0]:
df = spark.read.format("json")\
            .option("inferschema","true")\
            .option("multiline","true")\
            .load("/FileStore/tables/file5.json")

In [0]:
df.show()

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17580142}, b9...|          943|           20|            1|  null|
|null|   null|                  []|            0|            0|  

In [0]:
df.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
df = df.select("*",explode("restaurants").alias("new_rest")).drop("restaurants")

In [0]:
df.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_rest: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_table_book

In [0]:
df.select("new_rest.restaurant.cuisines","new_rest.restaurant.currency","new_rest.restaurant.name").show(truncate = False)

+---------------------------------+--------+------------------------------------+
|cuisines                         |currency|name                                |
+---------------------------------+--------+------------------------------------+
|Southern, Cajun, Soul Food       |$       |The Coop                            |
|Italian                          |$       |Maggiano's Little Italy             |
|Asian, Latin American, Vegetarian|$       |Tako Cheena by Pom Pom              |
|Mediterranean, Turkish           |$       |Bosphorous Turkish Cuisine          |
|Caribbean                        |$       |Bahama Breeze Island Grille         |
|Asian, Thai                      |$       |Hawkers Asian Street Fare           |
|American                         |$       |Seasons 52 Fresh Grill              |
|Irish                            |$       |Raglan Road Irish Pub and Restaurant|
|                                 |$       |Hillstone                           |
|German         

In [0]:
df = df.select("*",explode_outer("new_rest.restaurant.establishment_types").alias("est_types"))

In [0]:
df.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_rest: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_table_book

In [0]:
df = df.select("est_types").show()

+---------+
|est_types|
+---------+
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
+---------+
only showing top 20 rows

